### Imports

In [27]:
import numpy as np
import pandas as pd
import fastwer
import os

### Functions

In [28]:
import numpy as np

def calculate_iou(boxes1, boxes2):
    """Calculate intersection over union (IOU) between two sets of bounding boxes"""
    boxes1 = np.array(boxes1)
    boxes2 = np.array(boxes2)

    x1 = np.maximum(boxes1[:, 0], boxes2[:, 0])
    y1 = np.maximum(boxes1[:, 1], boxes2[:, 1])
    x2 = np.minimum(boxes1[:, 0] + boxes1[:, 2], boxes2[:, 0] + boxes2[:, 2])
    y2 = np.minimum(boxes1[:, 1] + boxes1[:, 3], boxes2[:, 1] + boxes2[:, 3])

    intersection = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)
    area1 = boxes1[:, 2] * boxes1[:, 3]
    area2 = boxes2[:, 2] * boxes2[:, 3]
    union = area1 + area2 - intersection
    iou = intersection / union

    return iou

def calculate_metrics(ground_truth, predictions, iou_threshold):
    """Calculate mAP, precision, recall, and f1score given ground truth and predicted bounding boxes"""
    ground_truth = np.array(ground_truth)
    predictions = np.array(predictions)

    iou = calculate_iou(ground_truth, predictions)
    iou_above_threshold = iou >= iou_threshold

    true_positives = set()
    false_positives = set(range(predictions.shape[0]))
    false_negatives = set(range(ground_truth.shape[0]))

    while np.any(iou_above_threshold):
        gt_idx, pred_idx = np.unravel_index(np.argmax(iou), iou.shape)
        if gt_idx not in false_negatives and pred_idx not in false_positives:
            true_positives.add((gt_idx, pred_idx))
        iou_above_threshold[gt_idx, :] = False
        iou_above_threshold[:, pred_idx] = False

    tp_count = len(true_positives)
    fp_count = predictions.shape[0] - tp_count
    fn_count = ground_truth.shape[0] - tp_count

    precision = tp_count / (tp_count + fp_count)
    recall = tp_count / (tp_count + fn_count)
    f1score = 2 * precision * recall / (precision + recall)
    average_iou = np.sum(np.max(iou, axis=1)) / iou.shape[0]

    return precision, recall, f1score, average_iou

In [29]:
pred_dir = './../../results/ocr/dbnet_vgg16/txt/'
gt_dir = './../../data/ocr/docbank/txt/'

In [30]:
for file in os.listdir(pred_dir):
    df = pd.read_csv(pred_dir + file, sep=' ')
    gt_df = pd.read_csv(gt_dir + file, sep=' ')

    preds = df[['X1','Y1','X2','Y2']].values.tolist()
    gts =  gt_df[['X1','Y1','X2','Y2']].values.tolist()
    print(calculate_metrics(gts, preds, 0.7))

ValueError: operands could not be broadcast together with shapes (446,) (337,) 

In [32]:
len(gts)

446

In [26]:
gt_df

,block,line,confidence,X1,Y1,X2,Y2,token
0,0,0,0.999791,1542,75,1566,109,9
1,1,0,0.999841,176,182,251,210,where
2,1,0,0.999616,253,178,302,214,the
3,1,0,0.999145,302,182,411,216,operator
4,1,0,0.970303,413,180,444,212,D
...,...,...,...,...,...,...,...,...
332,27,0,0.514569,825,1971,927,2005,u).a)
333,27,0,0.995181,925,1973,956,2003,+
334,27,0,0.571552,956,1971,1114,2003,(Vu):a-
335,27,0,0.486981,1111,1971,1246,2005,"Va:a),"


In [4]:
hypo = ['This is an example .', 'This is another example .']
ref = ['This is the example :)', 'That is the example .']

# Corpus-Level WER: 40.0
print(fastwer.score(hypo, ref))
# Corpus-Level CER: 25.5814
print(fastwer.score(hypo, ref, char_level=True))

# Sentence-Level WER: 40.0
print(fastwer.score_sent(hypo[0], ref[0]))
# Sentence-Level CER: 22.7273
print(fastwer.score_sent(hypo[0], ref[0], char_level=True))

40.0
25.5814
40.0
22.7273


In [ ]:
def calculate_iou(box1, box2):
    """Calculate intersection over union (IOU) of two bounding boxes"""
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    area1 = w1 * h1
    area2 = w2 * h2
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1 + w1, x2 + w2)
    y_bottom = min(y1 + h1, y2 + h2)
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    union_area = area1 + area2 - intersection_area
    return intersection_area / union_area

def calculate_metrics(ground_truth, predictions, iou_threshold):
    """Calculate mAP, precision, recall, and f1score given ground truth and predicted bounding boxes"""
    tp, fp, fn = 0, 0, 0
    iou_sum = 0.0
    for gt_box in ground_truth:
        iou_max = 0.0
        for pred_box in predictions:
            iou = calculate_iou(gt_box, pred_box)
            if iou >= iou_threshold and iou > iou_max:
                tp += 1
                iou_sum += iou
                iou_max = iou
            elif iou >= iou_threshold:
                fp += 1
        if iou_max < iou_threshold:
            fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1score = 2 * precision * recall / (precision + recall)
    if tp == 0:
        average_iou = 0.0
    else:
        average_iou = iou_sum / tp
    return precision, recall, f1score, average_iou


In [ ]:
import numpy as np

def calculate_iou(boxes1, boxes2):
    """Calculate intersection over union (IOU) between two sets of bounding boxes"""
    boxes1 = np.array(boxes1)
    boxes2 = np.array(boxes2)

    x1 = np.maximum(boxes1[:, 0], boxes2[:, 0])
    y1 = np.maximum(boxes1[:, 1], boxes2[:, 1])
    x2 = np.minimum(boxes1[:, 0] + boxes1[:, 2], boxes2[:, 0] + boxes2[:, 2])
    y2 = np.minimum(boxes1[:, 1] + boxes1[:, 3], boxes2[:, 1] + boxes2[:, 3])

    intersection = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)
    area1 = boxes1[:, 2] * boxes1[:, 3]
    area2 = boxes2[:, 2] * boxes2[:, 3]
    union = area1 + area2 - intersection
    iou = intersection / union

    return iou

def calculate_metrics(ground_truth, predictions, iou_threshold):
    """Calculate mAP, precision, recall, and f1score given ground truth and predicted bounding boxes"""
    ground_truth = np.array(ground_truth)
    predictions = np.array(predictions)

    iou = calculate_iou(ground_truth, predictions)
    tp = 0
    fp = 0
    fn = 0
    iou_sum = 0.0

    for i in range(len(ground_truth)):
        iou_gt = iou[i]
        tp_pred = np.sum(iou_gt >= iou_threshold)
        fp_pred = np.sum(iou_gt < iou_threshold)
        tp += tp_pred
        fp += fp_pred

        if tp_pred == 0:
            fn += 1
        else:
            iou_sum += np.max(iou_gt)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1score = 2 * precision * recall / (precision + recall)
    if tp == 0:
        average_iou = 0.0
    else:
        average_iou = iou_sum / tp

    return precision, recall, f1score, average_iou
